In [ ]:
# |default_exp whisper

# Constantes

In [ ]:
# |export

AUDIO_PATH = "audios"

# list mp3 files

In [ ]:
# |export

from download import get_audio_path
import os, glob


def list_mp3_files(audio_path=AUDIO_PATH):
    fullpath = get_audio_path(audio_path)

    return glob.glob(os.path.join(fullpath, "*.mp3"))

In [ ]:
list_mp3_files()[:5]

['/home/guillaume/git/lmelp/audios/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3',
 '/home/guillaume/git/lmelp/audios/14007-01.12.2024-ITEMA_23942372-2024F4007S0336-22.mp3',
 '/home/guillaume/git/lmelp/audios/14007-10.11.2024-ITEMA_23920569-2024F4007S0315-22.mp3']

# extract whisper

In [ ]:
# |export

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


def extract_whisper(mp3_filename):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )

    dataset = load_dataset(
        "distil-whisper/librispeech_long", "clean", split="validation"
    )
    sample = dataset[0]["audio"]

    result = pipe(
        mp3_filename,
        return_timestamps=True,
    )

    return result["text"]

In [ ]:
mp3_1 = list_mp3_files()[0]

whisper = extract_whisper(mp3_1)

whisper[:150]

Device set to use cpu
/usr/share/libdrm/amdgpu.ids: No such file or directory
amdgpu: unknown (family_id, chip_external_rev): (148, 10)
libGL error: failed to create dri screen
libGL error: failed to load driver: radeonsi
/home/guillaume/miniforge3/envs/whisper/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_ke

{'text': " France Inter Le masque et la plume Cinq romans sont à l'affiche du masque de ce jour avec une louve garou, la figure du couple Rob Grillet, le milieu de la finance dans l'Amérique des années 80 ou encore un super-héros baptisé AVC Mann. Face au public qui compte des étudiants de l'université Paris-Sorbonne, non j'ai dit dans le désordre, de l'université Sorbonne-Paris 4, sont installés sur la scène du Théâtre de l'Alliance française, Laurent Chalumeau, auteur, Raphaël Léris du journal Le Monde, Jean-Marc Proust du magazine en ligne Slate et Patricia Martin que vous retrouvez pour leur philo le vendredi à 20h et le dimanche à 14h. Plus précisément, dans la bibliothèque du masque aujourd'hui figurent les nouveaux romans de Marc Dugain, d'Emmanuel Lambert, de Mathieu Palin, de Fabrice Carreau et de la dessinatrice américaine Emile Ferris. Autant d'idées de cadeaux, ou pas, pour ce qui nous attend dans 48 heures. Bonjour et bienvenue, c'est le masque et ses plumes. Accords et dé

# store whisper in db 

In [ ]:
# |export

from bson import ObjectId


def store_whisper_in_db(whisper, collection, oid, force=False, verbose=False):
    """
    whisper: str, la transcription du fichier audio
    collection: pymongo collection
    oid: str, l'identifiant de l episode
    force: bool, si True, on ecrase le whisper existant

    return True si le whisper a ete stocke, False sinon
    """

    # Récupération du document
    document_entry = collection.find_one({"_id": ObjectId(oid)})

    if document_entry is None:
        if verbose:
            print(f"Document avec l'oid {oid} non trouvé")
        return False

    if "whisper" in document_entry and not force:
        if verbose:
            print(
                f"Whisper déjà stocké pour l'oid {oid}m et on ne force pas le stockage"
            )
        return False
    else:
        document_entry["whisper"] = whisper
        collection.update_one({"_id": ObjectId(oid)}, {"$set": document_entry})
        if verbose:
            print(f"Whisper stocké pour l'oid {oid}")
        return True

In [ ]:
from connection import get_collection

col = get_collection()
oid = "6773e32258fc5717f3516b98"
store_whisper_in_db("test whisper", col, oid, force=True, verbose=True)

Whisper stocké pour l'oid 6773e32258fc5717f3516b98


True

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("09 whisper mp3.ipynb", ".")